# Combine the relevant data sets
Data sets we can use:
    1. IMDB
    2. The Movie Database
    3. The Numbers

In [2]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import ttest_rel

df_numbers = pd.read_csv("../CleanedData/cleaned_numbers.csv")
df_numbers.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,production_budget_num,domestic_gross_num,worldwide_gross_num,domestic_ratio,worldwide_ratio,profit_num,roi_num,domestic_pct
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",425000000.0,760507625.0,2.776345e+09,1.789430,6.532577,2.351345e+09,5.532577,27.392401
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",410600000.0,241063875.0,1.045664e+09,0.587101,2.546673,6.350639e+08,1.546673,23.053668
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",350000000.0,42762350.0,1.497624e+08,0.122178,0.427892,-2.002376e+08,-0.572108,28.553472
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",330600000.0,459005868.0,1.403014e+09,1.388403,4.243841,1.072414e+09,3.243841,32.715702
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",317000000.0,620181382.0,1.316722e+09,1.956408,4.153696,9.997217e+08,3.153696,47.100413


In [3]:
df_imtm = pd.read_csv("../CleanedData/combinedimdbtmdb.csv")
df_imtm.head()

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,aka_title,region,language,...,id,original_language,original_title,popularity,title,vote_average,vote_count,release_year,log_popularity,combined_score
0,tt1375666,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066.0,A Origem,BR,Unknown,...,27205,en,Inception,27.920,Inception,8.3,22186,2010,3.364533,18042.8188
1,tt1345836,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769.0,A sötét lovag: Felemelkedés,HU,Unknown,...,49026,en,The Dark Knight Rises,26.220,The Dark Knight Rises,7.7,13933,2012,3.303952,12730.1006
2,tt0816692,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334.0,Bein kokhavim,IL,he,...,157336,en,Interstellar,28.440,Interstellar,8.2,18597,2014,3.382354,12699.2264
3,tt1853728,Django Unchained,2012,165.0,"Drama,Western",8.4,1211405.0,Django,PL,Unknown,...,68718,en,Django Unchained,21.260,Django Unchained,8.0,15725,2012,3.102791,11433.8020
4,tt0848228,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655.0,Angaarey Bane Sholey,IN,hi,...,24428,en,The Avengers,50.289,The Avengers,7.6,19673,2012,3.937476,11082.7535


In [3]:
# Quick exact title join using the titles for initial corelation checks
merged_check = pd.merge(df_numbers, df_imtm, left_on="movie", right_on="primary_title", how="inner")
merged_check.head()


,id_x,release_date,movie,production_budget,domestic_gross,worldwide_gross,production_budget_num,domestic_gross_num,worldwide_gross_num,domestic_ratio,...,id_y,original_language,original_title,popularity,title,vote_average,vote_count,release_year,log_popularity,combined_score
0,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",410600000.0,241063875.0,1.045664e+09,0.587101,...,1865,en,Pirates of the Caribbean: On Stranger Tides,30.579,Pirates of the Caribbean: On Stranger Tides,6.4,8571,2011,3.452492,3502.8624
1,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",330600000.0,459005868.0,1.403014e+09,1.388403,...,99861,en,Avengers: Age of Ultron,44.383,Avengers: Age of Ultron,7.3,13457,2015,3.815138,5841.1972
2,7,2018-04-27,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",300000000.0,678815482.0,2.048134e+09,2.262718,...,299536,en,Avengers: Infinity War,80.773,Avengers: Infinity War,8.3,13948,2018,4.403947,6860.5550
3,9,2017-11-17,Justice League,"$300,000,000","$229,024,295","$655,945,209",300000000.0,229024295.0,6.559452e+08,0.763414,...,141052,en,Justice League,34.953,Justice League,6.2,7510,2017,3.582213,2604.9975
4,10,2015-11-06,Spectre,"$300,000,000","$200,074,175","$879,620,923",300000000.0,200074175.0,8.796209e+08,0.666914,...,206647,en,Spectre,30.318,Spectre,6.4,6719,2015,3.444193,2827.0432


In [4]:
merged_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_x                   1480 non-null   int64  
 1   release_date           1480 non-null   object 
 2   movie                  1480 non-null   object 
 3   production_budget      1480 non-null   object 
 4   domestic_gross         1480 non-null   object 
 5   worldwide_gross        1480 non-null   object 
 6   production_budget_num  1480 non-null   float64
 7   domestic_gross_num     1480 non-null   float64
 8   worldwide_gross_num    1480 non-null   float64
 9   domestic_ratio         1480 non-null   float64
 10  worldwide_ratio        1480 non-null   float64
 11  profit_num             1480 non-null   float64
 12  roi_num                1480 non-null   float64
 13  domestic_pct           1445 non-null   float64
 14  movie_id               1480 non-null   object 
 15  prim

In [ ]:
#Use fuzzy match to compare titles

# Create a dictionary mapping  combined data title -> numbers_title()
title_map = {}

tmdb_titles = tmdb_df['title'].tolist()

for imdb_title in imdb_df['primary_title']:
    match, score, idx = process.extractOne(
        imdb_title, tmdb_titles, scorer=fuzz.token_sort_ratio
    )
    if score >= 85:  # threshold (0-100), adjust if too strict/loose
        title_map[imdb_title] = match


imdb_df['tmdb_match'] = imdb_df['primary_title'].map(title_map)

# Merge on imdb.primary_title -> tmdb.title
combined_df = imdb_df.merge(
    tmdb_df,
    left_on='tmdb_match',
    right_on='title',
    how='inner',
    suffixes=('_imdb','_tmdb')
)

In [11]:
#Use fuzzy match to compare titles

# Create a dictionary mapping imdb_title -> best tmdb_title
title_map = {}

numbers_titles = df_numbers['movie'].tolist() #List of all movie titles from numbers for cross reference

for imtm_title in df_imtm['primary_title']:
    match, score, idx = process.extractOne(
        imtm_title, numbers_titles, scorer=fuzz.token_sort_ratio
    )
    if score >= 85:  # threshold (0-100), adjust if too strict/loose
        title_map[imtm_title] = match


df_imtm['movie_name_match'] = df_imtm['primary_title'].map(title_map)

# Merge on imdb.primary_title -> tmdb.title
combined_df = df_imtm.merge(
    df_numbers,
    left_on='movie_name_match',
    right_on='movie',
    how='inner',
    suffixes=('_numbers','_imdb/tmdb')
)

In [12]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movie_id               1652 non-null   object 
 1   primary_title          1652 non-null   object 
 2   start_year             1652 non-null   int64  
 3   runtime_minutes        1652 non-null   float64
 4   genres                 1652 non-null   object 
 5   averagerating          1652 non-null   float64
 6   numvotes               1652 non-null   float64
 7   aka_title              1652 non-null   object 
 8   region                 1652 non-null   object 
 9   language               1652 non-null   object 
 10  log_numVotes           1652 non-null   float64
 11  runtime_winsorized     1652 non-null   float64
 12  tmdb_match             1652 non-null   object 
 13  Unnamed: 0             1652 non-null   int64  
 14  genre_ids              1652 non-null   object 
 15  id_n

In [13]:
combined_df.to_csv("finalcombined.csv", index=False)